In [1]:
# Enable auto-reloading of external modules - useful during development
%load_ext autoreload
%autoreload 2

# Configure Python path to find our custom modules
import sys
from pathlib import Path

# Add project root to the Python path for proper imports
project_root = Path.cwd().parent
if project_root not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
# Import necessary libraries
import src.processing as processing
import src.config as lists

In [3]:
# Load and process data
df = processing.load_data("/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv")
df_prepared = processing.prepare_data(df)
df_added_features = processing.create_all_model_features_orchestrated(df_prepared)
df_missing = processing.drop_missing_final_vars_streamlined(df_added_features, lists.final_set_A_predictor_names_and_dependent)
df_final = processing.annual_winsorize_variables(df_missing, lists.columns_to_winsorize)

/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/src/processing.py:42: DtypeWarning: Columns (10,11,12,16,17,22,26,30,33,34,35,945,946,948,949,950,955,969,970,975,980) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Data loaded successfully from /Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv
Original number of observations: 317304
Number of columns after selection: 30
Observations after year filter (2000-2023): 302751
Observations after excluding financial and utility firms: 170598
Starting feature construction. Initial df shape: (170598, 30)
  Creating lags for: ['at', 'ni', 'rect', 'invt', 'ap', 'sale']

Performing pre-calculation validity checks & preparations...
  Missing 'xrd' values filled with 0.
  'ipo_year' created from 'ipodate'.

Constructing dependent variable...
  OCF_Scaled_t_plus_1 created.

Constructing Set A (OLS) predictors...
  Set A predictors constructed.

Constructing control dummy variables...
  Dummy variables constructed.

Constructing Set B (additional ML) predictors...
  Set B predictors constructed.

Selecting final model variables and dropping intermediate columns...
  Shape of DataFrame 

In [4]:
# Split data chronologically - train on pre-2018, test on 2018+
train_df, test_df = processing.split_data_chronologically(df_final, 'fyear', split_year=2018)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Training period: {train_df['fyear'].min()} - {train_df['fyear'].max()}")
print(f"Test period: {test_df['fyear'].min()} - {test_df['fyear'].max()}")

Training set: 102567 obs (Predictor years <= 2018)
Test set: 20882 obs (Predictor years > 2018)
Training data shape: (102567, 30)
Test data shape: (20882, 30)
Training period: 2001.0 - 2018.0
Test period: 2019.0 - 2022.0


In [5]:
# Prepare features and dependent variable for Random Forest regression
X_train = train_df[lists.SET_A_FEATURES + lists.CONTROL_DUMMY_FEATURES]
y_train = train_df[lists.DEPENDENT_VARIABLE]

X_test = test_df[lists.SET_A_FEATURES + lists.CONTROL_DUMMY_FEATURES]
y_test = test_df[lists.DEPENDENT_VARIABLE]

# Drop dummy variables that caused multicollinearity in OLS
X_train = X_train.drop(columns=['ASC842_dummy', 'COVID_dummy'])
X_test = X_test.drop(columns=['ASC842_dummy', 'COVID_dummy'])

print("Features included in the model:")
for i, feature in enumerate(X_train.columns, 1):
    print(f"{i:2d}. {feature}")

print(f"\nDependent variable: {lists.DEPENDENT_VARIABLE}")
print(f"Training observations: {len(X_train)}")
print(f"Test observations: {len(X_test)}")

Features included in the model:
 1. OCF_Scaled_Lag_t
 2. NI_Scaled_t
 3. Accruals_Scaled_t
 4. Delta_Rec_Scaled_t
 5. Delta_Inv_Scaled_t
 6. Delta_AP_Scaled_t
 7. DP_Scaled_t
 8. ln_at_t
 9. ASC606_dummy
10. TCJA_dummy

Dependent variable: OCF_Scaled_t_plus_1
Training observations: 102567
Test observations: 20882


In [6]:
# =============================================================================
# BAYESIAN CROSS-VALIDATION FOR HYPERPARAMETER TUNING
# =============================================================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import numpy as np
import time

# Define hyperparameter search space for Bayesian optimization
search_space = {
    'n_estimators': Integer(50, 500),
    'max_depth': Integer(3, 30),
    'min_samples_split': Integer(2, 20),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Categorical(['sqrt', 'log2', None])
}

# Create Random Forest regressor
rf_regressor = RandomForestRegressor(random_state=42, n_jobs=-1)

# Perform Bayesian Search with 5-fold cross-validation
print("Performing Bayesian optimization with 5-fold cross-validation for Random Forest hyperparameter tuning...")
print("Using Gaussian Process to intelligently search hyperparameter space...")

start_time = time.time()

bayes_search = BayesSearchCV(
    estimator=rf_regressor,
    search_spaces=search_space,
    cv=5,
    scoring='r2',
    n_iter=50,  # Number of parameter settings sampled
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# Fit the Bayesian search
bayes_search.fit(X_train, y_train)

# Print best parameters and score
print(f"\nBest hyperparameters: {bayes_search.best_params_}")
print(f"Best cross-validation R² score: {bayes_search.best_score_:.4f}")

# Calculate standard deviation from cross-validation results
cv_scores = cross_val_score(bayes_search.best_estimator_, X_train, y_train, cv=5, scoring='r2')
print(f"Standard deviation: {np.std(cv_scores):.4f}")

elapsed_time = time.time() - start_time
print(f"Bayesian optimization completed in {elapsed_time:.2f} seconds")

Performing Bayesian optimization with 5-fold cross-validation for Random Forest hyperparameter tuning...
Using Gaussian Process to intelligently search hyperparameter space...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [ ]:
# =============================================================================
# RANDOM FOREST MODEL - For ML Model Comparisons
# =============================================================================

# Use the best model from Bayesian search
best_rf_model = bayes_search.best_estimator_

# Predictions
y_train_pred = best_rf_model.predict(X_train)
y_test_pred = best_rf_model.predict(X_test)

# Simple metrics function for any model
def print_model_performance(y_true, y_pred, model_name, dataset):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} ({dataset}): R²={r2:.4f} | RMSE={rmse:.4f} | MAE={mae:.4f}")

# Print results
print("RANDOM FOREST PERFORMANCE:")
print_model_performance(y_train, y_train_pred, "Random Forest", "Train")
print_model_performance(y_test, y_test_pred, "Random Forest", "Test")

In [ ]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

import pandas as pd

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("FEATURE IMPORTANCE RANKING:")
print("="*50)
for i, (_, row) in enumerate(feature_importance.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

# Print model details
print(f"\nMODEL DETAILS:")
print(f"N Estimators: {best_rf_model.n_estimators}")
print(f"Max Depth: {best_rf_model.max_depth}")
print(f"Min Samples Split: {best_rf_model.min_samples_split}")
print(f"Min Samples Leaf: {best_rf_model.min_samples_leaf}")
print(f"Max Features: {best_rf_model.max_features}")
print(f"Number of Features: {best_rf_model.n_features_in_}")
print(f"Out-of-bag Score: {best_rf_model.oob_score_ if hasattr(best_rf_model, 'oob_score_') and best_rf_model.oob_score else 'Not available'}")

In [ ]:
# =============================================================================
# ADDITIONAL RANDOM FOREST ANALYSIS
# =============================================================================

# Tree statistics
print("ENSEMBLE STATISTICS:")
print("="*50)
print(f"Number of trees in forest: {len(best_rf_model.estimators_)}")
print(f"Bootstrap samples: {best_rf_model.bootstrap}")
print(f"Random state: {best_rf_model.random_state}")

# Analyze individual tree depths
tree_depths = [estimator.get_depth() for estimator in best_rf_model.estimators_]
print(f"\nTREE DEPTH STATISTICS:")
print(f"Average tree depth: {np.mean(tree_depths):.2f}")
print(f"Min tree depth: {np.min(tree_depths)}")
print(f"Max tree depth: {np.max(tree_depths)}")
print(f"Std tree depth: {np.std(tree_depths):.2f}")

# Analyze number of leaves
tree_leaves = [estimator.get_n_leaves() for estimator in best_rf_model.estimators_]
print(f"\nTREE LEAVES STATISTICS:")
print(f"Average leaves per tree: {np.mean(tree_leaves):.2f}")
print(f"Min leaves per tree: {np.min(tree_leaves)}")
print(f"Max leaves per tree: {np.max(tree_leaves)}")
print(f"Std leaves per tree: {np.std(tree_leaves):.2f}")